In [2]:
import ipyparallel as ipp
c = ipp.Client()

In [3]:
c.ids

[0, 1, 2, 3]

In [4]:
c

In [5]:
c[:]

<DirectView [0, 1, 2, 3]>

In [7]:
c[0].apply_sync(lambda : "Hola alumnos")

'Hola alumnos'

In [8]:
c[:].apply_sync(lambda : 'hola')

['hola', 'hola', 'hola', 'hola']

In [9]:
dview = c[:]
dview.apply_sync(lambda : "Hola Alan")

['Hola Alan', 'Hola Alan', 'Hola Alan', 'Hola Alan']

In [10]:
dview

<DirectView [0, 1, 2, 3]>

In [16]:
dview = c.direct_view()
dview.apply_sync(lambda : "Hola, segundo día")

['Hola, segundo d\xc3\xada',
 'Hola, segundo d\xc3\xada',
 'Hola, segundo d\xc3\xada',
 'Hola, segundo d\xc3\xada']

In [18]:
import os
dview.apply_sync(os.getpid) 
#ids o hostname de la maquina donde reside cada motor

[9160, 2756, 9624, 5248]

In [19]:
import socket
dview.apply_sync(socket.gethostname)

['Yosua-PC', 'Yosua-PC', 'Yosua-PC', 'Yosua-PC']

1. Una vista proporciona acceso a un subconjunto de motores disponibles para el cliente.
2. Las tareas se asignan a los motores usando la vista
3. DirectView permite enviar tareas de manera explicita a un motor especifico
4. LoadBalancedView administra la distribucion de tareas y es útil cuando se tienen muchas tareas que toman diferentes tiempos para ejecutarse



In [45]:
from time import time
tiempoInicial = time()
print dview.map_sync (lambda x, y, z: x+y+z, range(10), range(10), range(10))
tiempoFinal = time()
tiempoEjecucion = tiempoInicial - tiempoFinal
print 'tiempo ejecución ',tiempoEjecucion


[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]
tiempo ejecución  -0.140000104904


In [40]:
from time import time
tiempoInicial = time()
c[1:3].map_sync(lambda x, y, z: x+y+z, range(10), range(10), range(10))
tiempoFinal = time()
tiempoEjecucion = tiempoInicial - tiempoFinal
print 'tiempo ejecución ',tiempoEjecucion

tiempo ejecución  -0.0780000686646


In [23]:
c[2].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [46]:
import numpy as np
lview = c.load_balanced_view()

In [49]:
lview.map_sync(lambda x: sum(x), np.random.random((10,1000000)))

[499677.54176492267,
 500301.96650435077,
 500114.40237124509,
 499922.29771281139,
 500112.58440728043,
 499742.82692879211,
 499965.70314221468,
 499329.93319386151,
 499865.04967150884,
 500231.87005297671]

In [50]:
def cube(x):
    return x**3
cube(2)

8

In [51]:
lista1 = [cube(x) for x in range(1,7)]
lista1

[1, 8, 27, 64, 125, 216]

In [55]:
map(cube, range(1,7))

[1, 8, 27, 64, 125, 216]

In [56]:
serial = map(cube, range(1,70))
parallel = dview.map_sync(cube, range(1,70))
serial == parallel

True

In [57]:
%timeit map(cube, range(1,70))
%timeit dview.map_sync(cube, range(1,70))

10000 loops, best of 3: 24.3 µs per loop
got unknown result: 06940c1e-b9c0-4800-b25c-3b037609a67d
10 loops, best of 3: 73.4 ms per loop


In [58]:
def multi(a,b):
    return a*b
dview.map_sync(multi, [0,1,3,5],[2,4,6,8])

[0, 4, 18, 40]

In [62]:
PrimeQ=lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:]) and x>1
print PrimeQ(2)
print PrimeQ(137)
print PrimeQ(135)

True
True
False


# Ejercicio
## Primos de Mersenne
### 1 y 2 : Función y selección de número primos.

In [2]:
PrimeQ=lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:]) and x>1
#PrimeQ(2**5-1)
for n in range(50):
    if PrimeQ(2**n-1) == True:
        print n

2
3
5
7
13
17
19
31


In [1]:
PrimeQ=lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:]) and x>1
#PrimeQ(2**5-1)
for n in range(60):
    if PrimeQ(2**n-1) == True:
        print n

2
3
5
7
13
17
19
31



KeyboardInterrupt



Solo llega a calcular hasta la iteración 50 de ahí en adelante la maquina empieza a consumir demasiada memoria, se realizao otra prueba con una iteración de 60, en donde el procesador (CPU) llego a un 59%, la memoria fisica a un 98% y el disco (Unidades Fisicas) a un 94%.
### 3: Estimación de tiempo en encontrar los números

In [37]:
for n in range(50):
    if PrimeQ(2**n-1) == True:
        print n 
%timeit PrimeQ(2**50-1)

2
3
5
7
13
17
19
31
1 loop, best of 3: 2.24 s per loop
